In [ ]:
!pip install openai -q

In [ ]:
import openai

openai.api_key = "YOUR API KEY"

In [ ]:
#The following cell is a v0 of the idea. It behaves well with GPT3.5-turbo so I decided to keep it.

In [ ]:
import json
import os

def generate_story_outline(logline):
    messages = [
        {"role": "system", "content": "You are a creative writing assistant with expertise in screenwriting and Truby's story structure. You are working on a short story so make sure the pace is fast."},
        {"role": "user", "content": f"Outline a short story based on this logline using Truby's story structure: {logline}. Include the hero's goal, the opponent, the conflict, the mentor, the theme, the hero's character change, and a big reveal or twist that will leave the audience speechless."}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=2048
    )

    story_outline = response['choices'][0]['message']['content']
    return story_outline

def generate_chapter_outlines(story_outline):
    messages = [
        {"role": "system", "content": "You are a creative writing assistant with expertise in screenwriting and Truby's story structure."},
        {"role": "user", "content": f"""Based on this short story outline: {story_outline}, generate short chapter outlines for the short story. Answer in JSON format where chapters are in a list inside the chapters attribute. It should look like this: {{
   "title": "the title",
   "author": "the author",
   "genre": "the genre",
   "chapters": [
      {{
         "chapter_number": 1,
         "chapter_title": "The title",
         "chapter_summary": "the summary"
      }}
   ]
}}"""}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=2048
    )

    chapter_outlines_str = response['choices'][0]['message']['content']
    novel_structure = json.loads(chapter_outlines_str)
    chapter_outlines = novel_structure["chapters"]
    print(chapter_outlines)
    return chapter_outlines

def refine_chapter_segment(segment, prior_messages):
    messages = prior_messages + [
        {"role": "system", "content": "You are a creative writing assistant that can rewrite text in the style of a New York Times bestseller."},
        {"role": "user", "content": f"Rewrite this text: {segment}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=2048
    )

    refined_segment = response['choices'][0]['message']['content']
    return refined_segment

def write_chapter(chapter_outline, logline, chapters):
    chapter = ""
    messages = [
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": f"Write a chapter for a short story based on this outline: {chapter_outline}. The logline of the story is this: {logline}. This is the full list of the chapters, but remember you are only writing one of them. {chapters} When the chapter is done write END CHAPTER"}
    ]
    while True:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=2048,
            stop=["END CHAPTER"]
        )

        content = response['choices'][0]['message']['content']
        refined_content = refine_chapter_segment(content, messages[-30:])
        chapter += refined_content
        #print(f"OUTLINE: {chapter_outline}")
        print("\n")
        print(refined_content)
        
        if "END CHAPTER" in chapter or len(chapter.split()) > 2000:
            print("CHAPTER IS FINISHED")
            break

        messages.append({"role": "user", "content": "Continue the chapter."})

    return chapter

def evaluate_and_refine_chapter(chapter):
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can evaluate and refine written work."},
        {"role": "user", "content": f"Evaluate and refine this chapter: {chapter}"}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=2048
    )

    refined_chapter = response['choices'][0]['message']['content']
    print(refined_chapter)
    return refined_chapter



def save_to_file(filename, content):
    with open(filename, "a") as f:
        f.write(content)

logline = "A teenager gets to meet her older self from the future"
story_outline = generate_story_outline(logline)
print(story_outline)
chapter_outlines = generate_chapter_outlines(story_outline)

output_filename = "short_story.txt"
if os.path.exists(output_filename):
    os.remove(output_filename)

short_story = ""
for outline in chapter_outlines:
    #print(outline)
    chapter = write_chapter(outline, logline, chapter_outlines)
    short_story += chapter
    save_to_file(output_filename, chapter)

print(short_story)



In [ ]:
# This part is a WIP of what could be a great GPT-4 version. More complex, maybe overly complex, but a good base.
#To be continued

In [ ]:
import json 
import os 

model = "gpt-4"

def generate_story_beats(logline):
    print("Generating Story Beats")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant with expertise in screenwriting and Truby's story structure. You are working on a short story so make sure the pace is fast."},
        {"role": "user", "content": f"Outline a short story based on this logline using Truby's story structure: {logline}. Include a big reveal or twist that will leave the audience speechless."}
      ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=2048)
    return response['choices'][0]['message']['content']

def refine_story_beats(story_beats):
    print("Refining Story Beats")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can refine story beats to the level of a New York Times bestseller."},
        {"role": "user", "content": f"You are working on a short story. Refine these story beats: {story_beats}"}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=2048)
    return response['choices'][0]['message']['content']

def generate_chapter_outlines(story_beats):
    print("Generating Chapters Outline")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant."},
        {"role": "user", "content": f"""Based on this short story outline: {story_beats}, generate short chapter outlines for the short story. Answer in JSON format where chapters are in a list inside the chapters attribute. It should look like this: {{
   "title": "the title",
   "author": "the author",
   "genre": "the genre",
   "chapters": [
      {{
         "chapter_number": 1,
         "chapter_title": "The title",
         "chapter_summary": "the summary"
      }}
   ]
}}"""}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=2048)
    chapter_outlines_str = response['choices'][0]['message']['content']
    novel_structure = json.loads(chapter_outlines_str)
    return novel_structure

def refine_chapter_outlines(chapter_outlines):
    print("Refining Chapters Outline")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can refine chapter outlines to the level of a New York Times bestseller."},
        {"role": "user", "content": f"Refine these chapter outlines in JSON format using double quotes: {chapter_outlines}"}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=2048)
    
    chapter_outlines_str = response['choices'][0]['message']['content']
    #print(chapter_outlines_str)
    novel_structure = json.loads(chapter_outlines_str)
    return novel_structure

def generate_context(outline, story_elements):
    print("Generating Context for the chapter")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can generate context for a short story."},
        {"role": "user", "content": f"Generate context based on this chapter outline: {outline} and previous story elements: {story_elements}"}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=2048)
    return response['choices'][0]['message']['content']

def write_chapter(outline, chapter_outlines, context, segments, logline):
    print("Writing the first draft of the chapter")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant writing the script for a short story. When the chapter is finished, write END CHAPTER."},
        {"role": "user", "content": f"Write a chapter for a short story based on this outline: {outline}. The logline of the story is this: {logline}. This is the full list of the chapters, but remember you are only writing one of them. {chapter_outlines}. Here is the context of the story {context} When the chapter is done write END CHAPTER"}
    ]
    
    for segment in segments[-5:]:
        messages.append({"role":"assistant", "content":segment})
        messages.append({"role":"user", "content":"continue this chapter. When the chapter is over write END CHAPTER"})
    #print(messages)
    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=4000, stop=["END CHAPTER"])
    return response['choices'][0]['message']['content']
def refine_chapter_segment(segment):
    print("Refining the draft of the chapter")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can rewrite text in the style of a New York Time Best Seller."},
        {"role": "user", "content": f"Rewrite this text: {segment}"}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=4000)
    print(response)
    return response['choices'][0]['message']['content']

def evaluate_and_review_chapter(chapter):
    print("Evaluating the quality of the story")
    messages = [
        {"role": "system", "content": "You are a creative writing assistant that can evaluate and review written work."},
        {"role": "user", "content": f"Evaluate and review this chapter: {chapter}"}
    ]

    response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=8000)
    return response['choices'][0]['message']['content']

logline = "A teenager gets to meet her older self from the future"

story_beats = generate_story_beats(logline)
refined_story_beats = refine_story_beats(story_beats)
chapter_outlines = generate_chapter_outlines(refined_story_beats)
refined_chapter_outlines = refine_chapter_outlines(chapter_outlines)

outline_filename = "chapters.txt"
output_filename = "short_story.txt"
review_filename = "review.txt"

if os.path.exists(outline_filename):
    os.remove(outline_filename)
if os.path.exists(output_filename):
    os.remove(output_filename)
if os.path.exists(review_filename):
    os.remove(review_filename)

def save_to_file(filename, content):
    with open(filename, "a") as f:
        f.write(content)
    
save_to_file(outline_filename, json.dumps(refined_chapter_outlines))
short_story = ""
for index, outline in enumerate(refined_chapter_outlines["chapters"]):
    print(f"Working on Chapter {index}")
    chapter = f"CHAPTER {refined_chapter_outlines['chapters'][index]['chapter_number']}: {refined_chapter_outlines['chapters'][index]['chapter_title']}"
    context = generate_context(outline, refined_chapter_outlines["chapters"][:index])
    segments = []
    while "END CHAPTER" not in chapter:
        segment = write_chapter(outline, refined_chapter_outlines, context, segments, logline)
        #print(segment)
        #segment = chapter.split()[-50:]
        refined_segment = refine_chapter_segment(segment)
        segments.append(segment)
        chapter += refined_segment
        if "END CHAPTER" in segment:
            #print("End of Chapter")
            #print(chapter)
            break

    chapter = chapter.replace("END CHAPTER", "")
    print(chapter)
    short_story += chapter
    save_to_file(output_filename, chapter)

    review = evaluate_and_review_chapter(chapter)
    save_to_file(review_filename, review)

print(short_story)